In [ ]:
mod_path = '../input/linknet-or-unet-results/'
import yaml
import pprint

THRESHOLD = 0.3 # preds > THRESHOLD
WINDOW = 1024
MIN_OVERLAP = 300

CHECKSUM = True
selected_models = [0, 1, 2, 3, 4]
# selected_models = []
torch_model_paths = [
    '../input/hubmap-model-cv/model_dh_cv5_0.pth',
    '../input/hubmap-model-cv/model_dh_cv5_2.pth',
]
TTAS = [0, 1, 2, 3]
VOTERS = 0.5
def flip(img, axis=0):
    if axis == 1:
        return img[::-1, :, ]
    elif axis == 2:
        return img[:, ::-1, ]
    elif axis == 3:
        return img[::-1, ::-1, ]
    else:
        return img

# Packages

In [ ]:
! pip install ../input/kerasapplications/keras-team-keras-applications-3b180cb -f ./ --no-index -q
! pip install ../input/efficientnet/efficientnet-1.1.0/ -f ./ --no-index -q
! pip install --no-index ../input/segmentation-models-pytorch/efficientnet_pytorch-0.6.3.tar.gz
! pip install --no-index ../input/segmentation-models-pytorch/pretrainedmodels-0.7.4.tar.gz
! pip install --no-index ../input/segmentation-models-pytorch/timm-0.3.2-py3-none-any.whl
! pip install --no-index ../input/segmentation-models-pytorch/segmentation_models_pytorch-0.1.3-py3-none-any.whl
! pip install --no-index ../input/segmentation-models-pytorch/typing_extensions-3.7.4.3-py3-none-any.whl

In [ ]:
import numpy as np
import pandas as pd
import os
import glob
import gc

import rasterio
from rasterio.windows import Window

import pathlib
from tqdm.notebook import tqdm
import cv2

import tensorflow as tf
import efficientnet as efn
import efficientnet.tfkeras

In [ ]:
import os, gc, json, cv2, tifffile, warnings, torch 

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors

from pathlib import Path
from torch.utils.data import Dataset as BaseDataset

warnings.simplefilter('ignore')

# Functions

In [ ]:
def rle_encode_less_memory(img):
    pixels = img.T.flatten()
    pixels[0] = 0
    pixels[-1] = 0
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 2
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

def make_grid(shape, window=256, min_overlap=32):
    """
        Return Array of size (N,4), where N - number of tiles,
        2nd axis represente slices: x1,x2,y1,y2 
    """
    x, y = shape
    nx = x // (window - min_overlap) + 1
    x1 = np.linspace(0, x, num=nx, endpoint=False, dtype=np.int64)
    x1[-1] = x - window
    x2 = (x1 + window).clip(0, x)
    ny = y // (window - min_overlap) + 1
    y1 = np.linspace(0, y, num=ny, endpoint=False, dtype=np.int64)
    y1[-1] = y - window
    y2 = (y1 + window).clip(0, y)
    slices = np.zeros((nx,ny, 4), dtype=np.int64)
    
    for i in range(nx):
        for j in range(ny):
            slices[i,j] = x1[i], x2[i], y1[j], y2[j]    
    return slices.reshape(nx*ny,4)

# Models

In [ ]:
identity = rasterio.Affine(1, 0, 0, 0, 1, 0)
fold_models = []
for selected_models_idx in selected_models:
    fold_model_path = f'../input/linknet-or-unet-results/model-fold-{selected_models_idx}.h5'
    print('keras:', fold_model_path)

    model = tf.keras.models.load_model(fold_model_path,compile = False)
    
    fold_models.append({
        'type': 'keras',
        'model': model,
        'size': 256,
    })

torch_models = [torch.load(path) for path in torch_model_paths]
for model in torch_models:
    fold_models.append({
        'type': 'torch',
        'model': model,
        'size': 512,
    })

print(len(fold_models))

# Tfrecords functions

In [ ]:
AUTO = tf.data.experimental.AUTOTUNE
image_feature = {
    'image': tf.io.FixedLenFeature([], tf.string),
    'x1': tf.io.FixedLenFeature([], tf.int64),
    'y1': tf.io.FixedLenFeature([], tf.int64)
}
def _parse_image(example_proto, tta_mode=0):
    example = tf.io.parse_single_example(example_proto, image_feature)
    image = tf.reshape( tf.io.decode_raw(example['image'],out_type=np.dtype('uint8')), (P['DIM'],P['DIM'], 3))
    
    # 左右
    if tta_mode == 2:
        image = tf.image.flip_left_right(image)
    # 上下
    if tta_mode == 1:
        image = tf.image.flip_up_down(image)
    # 反转
    if tta_mode == 3:
        image = tf.image.rot90(image, k=2)
        
    return image, example['x1'], example['y1']

def load_dataset(filenames, ordered=True, tta_mode=0):
    ignore_order = tf.data.Options()
    if not ordered:
        ignore_order.experimental_deterministic = False
    dataset = tf.data.TFRecordDataset(filenames)
    dataset = dataset.with_options(ignore_order)
    dataset = dataset.map(lambda ex: _parse_image(ex, tta_mode=tta_mode))
    return dataset

def get_dataset(FILENAME, tta_mode=0):
    dataset = load_dataset(FILENAME, tta_mode=tta_mode)
    dataset  = dataset.batch(64)
    dataset = dataset.prefetch(AUTO)
    return dataset

# Results

In [ ]:
p = pathlib.Path('../input/hubmap-kidney-segmentation')
subm = {}

for i, filename in tqdm(enumerate(p.glob('test/*.tiff')), 
                        total = len(list(p.glob('test/*.tiff')))):
    
    print(f'{i+1} Predicting {filename.stem}')
    
    dataset = rasterio.open(filename.as_posix(), transform = identity)
    preds = np.zeros(dataset.shape, dtype=np.uint8)    

    print('SUBMISSION_MODE: FULL')
    slices = make_grid(dataset.shape, window=WINDOW, min_overlap=MIN_OVERLAP)

    if dataset.count != 3:
        print('Image file with subdatasets as channels')
        layers = [rasterio.open(subd) for subd in dataset.subdatasets]
            
    for (x1,x2,y1,y2) in slices:
        if dataset.count == 3:
            image = dataset.read([1,2,3], window=Window.from_slices((x1,x2),(y1,y2)))
            image = np.moveaxis(image, 0, -1)
        else:
            image = np.zeros((WINDOW, WINDOW, 3), dtype=np.uint8)
            for fl in range(3):
                image[:,:,fl] = layers[fl].read(window=Window.from_slices((x1,x2),(y1,y2)))

        images = {
            '256': cv2.cvtColor(cv2.resize(image, (256, 256), interpolation=cv2.INTER_AREA), cv2.COLOR_RGB2BGR),
            '512': cv2.cvtColor(cv2.resize(image, (512, 512), interpolation=cv2.INTER_AREA), cv2.COLOR_RGB2BGR),
        }

        pred = np.zeros((WINDOW, WINDOW), dtype=np.float32)
        for fold_model  in fold_models:
            model = fold_model['model']
            if fold_model['type'] == 'keras':
                fold_image = images[str(fold_model['size'])]
                im_sh = fold_image.shape
                fold_pred = np.squeeze(model.predict(fold_image.reshape((1, im_sh[0], im_sh[1], im_sh[2]))))
            elif fold_model['type'] == 'torch':
                fold_image = images[str(fold_model['size'])]
                fold_image = fold_image.transpose(2, 0, 1).astype(np.float32) / 255.0
                
                fold_pred = model.predict(torch.from_numpy(fold_image).to('cuda').unsqueeze(0))
                fold_pred = fold_pred.squeeze().cpu().numpy()[1, :, :]

            fold_pred = cv2.resize(fold_pred, (WINDOW, WINDOW))
            pred += fold_pred

        pred = pred / len(fold_models)
        preds[x1:x2,y1:y2] += (pred > THRESHOLD).astype(np.uint8)

    preds = (preds > VOTERS).astype(np.uint8)
    
    contours, hierarchy = cv2.findContours(preds, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    areas = []
    c = np.zeros(preds.shape, np.uint8)
    for contour in contours:
        area = cv2.contourArea(contour)

        if area < 7000:
            c = cv2.drawContours(c, [contour], -1, 1, cv2.FILLED)

    preds = np.logical_and(preds, np.logical_not(c)).astype(np.uint8)
    del contours, c
    gc.collect()
    
    subm[i] = {'id':filename.stem, 'predicted': rle_encode_less_memory(preds)}
    
    if CHECKSUM:
        print('Checksum: '+ str(np.sum(preds)))
    
    del preds
    gc.collect();

# Making submission

In [ ]:
submission = pd.DataFrame.from_dict(subm, orient='index')
submission.to_csv('submission.csv', index=False)
submission.head()